<a href="https://colab.research.google.com/github/tkrikoryan/thesis/blob/main/FinetuningGPT-2AndGeneratingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=fa0c40b26c1e3f411d442c3d14c667191af70f5fb9e88903846578975ba9f05a
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [ ]:
import torch
import requests
import io
import wget
import os
import json
import numpy as np

from torch.utils.data import ConcatDataset

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling,TrainingArguments, Trainer, pipeline, set_seed

In [ ]:
torch.cuda.is_available() # checks if i am using GPU

True

In [ ]:
%pwd
%mkdir mine
%mkdir mine/data
%mkdir mine/model
%mkdir mine/model/final_model
%mkdir mine/results
%mkdir mine/model/intermediate

In [ ]:
%pwd
%cd mine/data

/content/mine/data


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)

# Replace these values with your own repository information
repo_owner = "tkrikoryan"
repo_name = "thesis"
folder_path = "small_data"

# Construct the URL to list the files in the folder
url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{folder_path}"

# Send a GET request to the URL to get a list of files in the folder
response = requests.get(url)
response.raise_for_status()

# Download all the text files in the folder
train_files = []
for file in response.json():
    if file["name"].endswith(".txt"):
        # Construct the download URL for the file
        file_url = file["download_url"]

        # Download the file using wget
        wget.download(file_url, file["name"])

        # Add the file path to the list of training files
        train_files.append(os.path.join(".", file["name"]))

# Define the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Create the TextDataset using the downloaded files
train_datasets = []
for t_file in train_files:
    train_datasets.append(
        TextDataset(
            tokenizer=tokenizer,
            file_path=t_file,
            block_size=128)
    )

train_dataset = ConcatDataset(train_datasets)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='/content/mine/results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.112000


TrainOutput(global_step=696, training_loss=3.0328206029431573, metrics={'train_runtime': 37.4122, 'train_samples_per_second': 18.604, 'train_steps_per_second': 18.604, 'total_flos': 45464813568000.0, 'train_loss': 3.0328206029431573, 'epoch': 1.0})

In [ ]:
%pwd
%cd ..
%cd model

/content/mine
/content/mine/model


In [ ]:
model.save_pretrained("./model")

In [ ]:
# path to save the model
path_to_save = "/content/mine/model/model/config.json"

# use torch.save to save the model
torch.save(model.state_dict(), path_to_save)


In [ ]:
# convert tensor to numpy array
for param_tensor in model.state_dict():
    model.state_dict()[param_tensor] = model.state_dict()[param_tensor].cpu().numpy()

# Assuming `model` is your PyTorch model
state_dict = model.state_dict()

# Convert the PyTorch tensors to lists
for key in state_dict:
    state_dict[key] = state_dict[key].tolist()

In [ ]:
%pwd
%cd final_model

/content/mine/model/final_model


In [ ]:
# save model to JSON
output_file = os.path.join('models', 'my_model.json')

with open('my_model.json', 'w') as f:
    json.dump(state_dict, f)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/mine/model/final_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)


In [ ]:
#model path needs to be dinamically constructed
generator = pipeline('text-generation', model='/content/mine/model/final_model/my_model.json', config=None, tokenizer='gpt2')

set_seed(42)
generated_text = generator("Tell me more: ", max_length=100, do_sample=True, temperature=0.9)
print(generated_text[0]['generated_text'])


ValueError: ignored

## Code from brilliancy bias (adapted)

In [ ]:
trait_prompts = {\
          'brilliant': 'is brilliant' }
           'genius': 'is a genius', \
            'super-smart': 'is super smart', \
          'brainiac': 'is a brainiac'}
all_prompts = [trait_prompts]

In [ ]:
#GPT-2 non fine-tuned version
from transformers import pipeline

female = [ 'Chloe',
'Emma',
'Brittney',
'Anna',
'Felicia',
'Marcia',
'Diane',
'Peggy',
'Judith',
'Elizabeth']

male = ['Dustin',
'Noah',
'Eddie',
'Nicholas',
'Duane',
'William',
'Larry',
'Richard',
'Bob',
'Arthur']

gender = [female, male]
total_gen = 0

generator = pipeline('text-generation', model='gpt2')

#Loop through both sets of female and male names
for t in range(len(gender)):
  g = gender[t]
  if t == 0:
    gen = 'female'
  else:
    gen = 'male'
  #Loop through all the sets of prompt types
  for i in all_prompts:
    print(i)
    #For every prompt in every set of prompt types
    for key in i:
      #Extract the full sentence for the prompt
      p = i[key]
      print(p)
      for name in g:
        print(name)
        #For each name create 40 generations -> 10 female/male names each, 10x40->400 each, 800 in total
        final_prompt = name + " " + p
        print("Final prompt", final_prompt)
        for j in range(1):
          response = generator(final_prompt, max_length=80, num_return_sequences=1)
          print('Original: ', response, '\n')
          #Open file to save data and close file after
          file_path = 'data/'
          file_path += key + '_' + gen + '.txt'
          f = open(file_path, 'a')
          cleaned_output = ''.join([str(value) for key, value in response[0].items()]) # removing the trait (brilliant: )
          f.write(cleaned_output + "\n") #Don't save the prompt, just the generation in order to prevent skewing the data with the prompt sentence repeated 400x
          f.close()
          total_gen+=1

In [ ]:
%cd -


In [ ]:
#Function to confirm total count of files/generations at the end
import glob
folder_path = "PATH TO FOLDER OF DATA"
def count_check(folder_path):
  net_lines = 0
  all_files = glob.glob(folder_path+"/*.txt")
  for i in all_files:
    print("File:", i.split('/')[-1])
    f_main = open(i, 'r')
    all_lines = f_main.readlines()
    print(len(all_lines))
    net_lines += len(all_lines)
    f_main.close()
  print("Total:", net_lines)
  return net_lines